## Classification with role description
This could be used after determing the first class (for which we have already achieved around 85% accuracy), to make decision about fine-grained roles.
We will give BERT the next input: [CLS] text with [START_SPAN] named entity [END_SPAN] text [SEP] role description, 
and perform a binary classification task over the contextalized embeddings whether a role describes named entity or not.

Intuitively this should be a better solution than just performing multilabel-classifiaction task over 12 or 6 classes, where we just present different classes with zeros and ones -> with given role description, we should know better if these classes describe the observed named entity.

### WIP

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')
# sub1 = 'drive/My Drive/Colab Notebooks/subtask1.parquet'
# print(sub1)

from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data'
sub1 = str(wd) + '/subtask1.parquet'
sub2 = str(wd) + '/description_roles.csv'
print(sub1)

c:\ml\semeval2025-t10\merged_data/subtask1.parquet


In [3]:
import pandas as pd
df = pd.read_parquet(sub1)
df_roles = pd.read_csv(sub2)

In [4]:
import re
def labelNum1(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text

df['label1'] = df.apply(labelNum1,axis=1)
df['input'] = df.apply(cleanText,axis=1)

def labelNum2(row):
    labels2 = [0 for _ in range(12)]
    if row['label1'] == 2:
        #labels2 = [0 for _ in range(6)]
        if 'Guardian' in row['classes2']:
            labels2[0] = 1
        if 'Martyr' in row['classes2']:
            labels2[1] = 1
        if 'Peacemaker' in row['classes2']:
            labels2[2] = 1
        if 'Rebel' in row['classes2']:
            labels2[3] = 1
        if 'Underdog' in row['classes2']:
            labels2[4] = 1
        if 'Virtuous' in row['classes2']:
            labels2[5] = 1
    elif row['label1'] == 0:
        #labels2 = [0 for _ in range(12)]
        if 'Instigator' in row['classes2']:
           labels2[0] = 1
        if 'Conspirator' in row['classes2']:
            labels2[1] = 1
        if 'Tyrant' in row['classes2']:
            labels2[2] = 1
        if  'Foreign Adversary' in row['classes2']:
            labels2[3] = 1
        if 'Traitor' in row['classes2']:
            labels2[4] = 1
        if 'Spy' in row['classes2']:
            labels2[5] = 1
        if 'Saboteur' in row['classes2']:
            labels2[6] = 1
        if 'Corrupt' in row['classes2']:
            labels2[7] = 1
        if 'Incompetent' in row['classes2']:
            labels2[8] = 1
        if 'Terrorist' in row['classes2']:
            labels2[9] = 1
        if 'Deceiver' in row['classes2']:
            labels2[10] = 1
        if 'Bigot' in row['classes2']:
            labels2[11] = 1
    elif row['label1'] == 1:
        #labels2 = [0 for _ in range(4)]
        if 'Forgotten' in row['classes2']:
            labels2[0] = 1
        if 'Exploited' in row['classes2']:
            labels2[1] = 1
        if 'Victim' in row['classes2']:
            labels2[2] = 1
        if 'Scapegoat' in row['classes2']:
            labels2[3] = 1
    return labels2

df['label2'] = df.apply(labelNum2, axis=1)

In [6]:
#to clean and precise start and end positions of NEs

def find_all_substring_start_end(text, substring):
    matches = re.finditer(re.escape(substring), text)
    positions = [(match.start(), match.end()) for match in matches]

    return positions

def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]

df['new_start_end'] = df.apply(adjust_start_end,axis=1)

# add SPAN_START and SPAN_END tokens to input
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)

# adjust again start and end positions
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

In [13]:
# prepare data for our model
df_roles
#...spojiti svaki fine-grained role i description sa retkom koji ima tu nadklasu, dodati IDjeve 1-4 ili 1-12 ili 1-6 po kojem cemo znati grupirati rjesenja binarne klasifikacije i pretociti u rjesenje multilabel klasifikacije

,Main role,fine-grained role,description
0,Protagonist,Guardian,Heroes or guardians who protect values or comm...
1,Protagonist,Martyr,Martyrs or saviors who sacrifice their well-be...
2,Protagonist,Peacemaker,"Individuals who advocate for harmony, working\..."
3,Protagonist,Rebel,"Rebels, revolutionaries, or freedom fighters w..."
4,Protagonist,Underdog,Entities who are considered unlikely to succee...
5,Protagonist,Virtuous,"Individuals portrayed as virtuous, righteous, ..."
6,Antagonist,Instigator,"Individuals or groups initiating conflict, oft..."
7,Antagonist,Conspirator,"Those involved in plots and secret plans, ofte..."
8,Antagonist,Tyrant,yrants and corrupt officials who abuse their p...
9,Antagonist,Foreign Adversary,Entities from other nations or regions creatin...


In [2]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaModel, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaModel.from_pretrained("xlm-roberta-base").to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

Using device: cpu


C:\Users\cvita\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
# kako cu dodati SEP tj </s> ...
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250004, 768, padding_idx=1)

In [22]:
data = df.loc[ : , ['span_input', 'label1', 'label2', 'new_start_end', 'entity']]
data

In [24]:
data['tokenized']=data.apply(preprocess_function,axis=1)

In [25]:
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [26]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))

2902 2902 2902


In [27]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i

    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])

In [28]:
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)

inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y1 = data['label1'].values
y2 = data['label2'].values

In [29]:
from sklearn.model_selection import train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y1_train, y1_test, y2_train, y2_test = train_test_split(
    inputIds_np, attMask_np, y1, y2, test_size=0.2, random_state=42, shuffle=True
)

In [30]:
import numpy as np
y2_train = np.array(y2_train.tolist(), dtype=np.int8)
y2_test = np.array(y2_test.tolist(), dtype=np.int8)

In [31]:
X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
y1_train = torch.tensor(y1_train, dtype=torch.long).to(device)
y1_test = torch.tensor(y1_test, dtype=torch.long).to(device)
y2_train = torch.tensor(y2_train, dtype=torch.long).to(device)
y2_test = torch.tensor(y2_test, dtype=torch.long).to(device)

In [32]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_ids, X_train_mask, y1_train, y2_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y1_test, y2_test )

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True) #shuffle=True provides data shuffle for batches in different epochs
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [33]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn

#first layer classifier
classifier = nn.Linear(model.config.hidden_size, 2).to(device)
optimizer = AdamW(model.parameters(), lr=8e-6)
criterion = nn.CrossEntropyLoss()